<a href="https://colab.research.google.com/github/Boukhlifi/VGG16/blob/main/VGG_extractot_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import Model 

In [34]:
#creation de modéle VGG mais en gardant que les couches d'extractions des caractéristiques
#cad que je vais enlever la derniere couche de fully connected qui contient 1000 neuronnes
#parcque le modele a été entrainé sur IMageNet qui contient 1000 classes
def create_model_descriptor():
  vgg_descriptor = VGG16()
  vgg_descriptor.layers.pop() #supression de quelques couches
  vgg_descriptor = Model(inputs = vgg_descriptor.inputs, outputs = vgg_descriptor.layers[-2].output)
  vgg_descriptor.summary()
  #create de VGG_imageNet without the last layer
  return vgg_descriptor

In [35]:
#faire des prétrétement sur l'image pourqu'elle soit compatible avec la basse de donnes imageNet qui été
#entrainné par le modele  VGG16
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(224,224))  #redimensionner l'image chargeé
  img = img_to_array(img)
  img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])#to shape (1,n,m,3) car imageNet est sous cette forme là
  img = preprocess_input(img)#pour convertir l'image RGB to BGR,et transmettre l'image a la gamme [-1,1](centrage à ZERo)
  return img 

In [36]:
#cette fonction va nous generer un vecteur de caractéristique de l'image d'entré
def get_image_representation(vgg_descriptor, img):
  return vgg_descriptor.predict(preprocess_image(img))[0,:]

In [50]:
#le calcule de la distance entre deux descripteurs
def findeuclidienDistance(source_representation, test_representation):
  euclidean_distance = source_representation - test_representation
  euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
  euclidean_distance = np.sqrt(euclidean_distance)
  return euclidean_distance

In [41]:
#cette fonction verifie si les deux images sont similaires ou non avec un taux d'acceptation epsilon
def verify(vgg_descriptor, img1, img2, epsilon=90):
  img1_representaion = get_image_representation(vgg_descriptor, img1)
  img2_representaion = get_image_representation(vgg_descriptor, img2)
  distance_euclidien = findeuclidienDistance(img1_representaion, img2_representaion)
  print("distance euclidien : ",distance_euclidien)
  if distance_euclidien < epsilon:
    print("they are the same objects")
  else:
    print("they are not the same objects")

In [39]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
if __name__ == "__main__":
  vgg_descriptor = create_model_descriptor()
  verify(vgg_descriptor, "/content/drive/My Drive/Colab Notebooks/mug1.jpg","/content/drive/My Drive/Colab Notebooks/mug2.jpg" , epsilon=90)
  features = get_image_representation(vgg_descriptor, "/content/drive/My Drive/Colab Notebooks/mug1.jpg")
  print("le descrpteur est de taille : ",features.shape)


Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [ ]:
r